In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from plotly.subplots import make_subplots
import plotly.express as px
import unidecode

In [ ]:
from formula_one import FormulaOne

In [ ]:
f1 = FormulaOne()
f1

In [ ]:
f1.update()

In [ ]:
fantasy_rosters = pd.read_csv('./draft/fantasy_rosters.csv', index_col=0)
fantasy_rosters

In [ ]:
(f1.results
 .tail(20)
 .merge(fantasy_rosters, on='DriverID')
 .loc[:, ['Points', 'Team', 'DriverID']]
)

In [ ]:
f1.results.groupby('RaceID').sum().loc[:, 'Points'].value_counts()

In [ ]:
f1.results

In [ ]:
s = f1.results.groupby('RaceID').sum().loc[:, 'Points']
s[s == 117]

In [ ]:
f1.results

In [ ]:
f1.results = f1.results.drop_duplicates().reset_index(drop=True)

In [ ]:
f1.results

In [ ]:
f1.save_data()

In [ ]:
f1.results[f1.results['RaceID'] == 202105]

In [ ]:
for i in range(202101, 202108):
    print(i, len(np.array(f1.results[f1.results['RaceID'] == i]['Points'])))

In [ ]:
pts_per_round = (
    f1.results.loc[f1.results['RaceID'] > 202100]
    .merge(fantasy_rosters[['Team', 'DriverID']], 
           on='DriverID')
    .merge(f1.races[['RaceID', 'Round']])
    .sort_values(['RaceID', 'Position'])
    .loc[:, ['Points', 'Team', 'Round']]
    .groupby(['Round', 'Team']).sum()
    .reset_index()
)

pts_per_round

In [ ]:
cumulative_points = (
    pts_per_round
    .groupby(['Team', 'Round']).sum()
    .groupby('Team').cumsum()
    .reset_index()
)
cumulative_points

In [ ]:
px.line(x='Round', y='Points', color='Team', data_frame=cumulative_points)

In [ ]:
with plt.style.context(['ggplot', 'seaborn-talk']):
    fig, (ax1, ax2) = plt.subplots(nrows=2, figsize=(12, 8))
    sns.lineplot(x='Round', y='Points', hue='Team', data=cumulative_points, ax=ax1)
    sns.barplot(x='Round', y='Points', hue='Team', data=pts_per_round, ax=ax2)
    fig.tight_layout()